In [ ]:
import numpy as np
import os
import csv
from scipy.stats import poisson
from scipy.stats import norm
import pandas as pd
import itertools
import matplotlib.pyplot as plot
%run -i utils.py

### Historical training

In [ ]:
fixture_list_1, fixture_list_2, fixture_list_3 = import_fixture_lists("../data/prem_results_20162017.csv",
                                                                      "../data/prem_results_20172018.csv",
                                                                      "../data/prem_results_20182019.csv")

In [ ]:
# all teams in three seasons and fourth season
team_list = np.concatenate((np.unique(fixture_list_1.as_matrix()[:, 0]),
                            np.unique(fixture_list_2.as_matrix()[:, 0]),
                            np.unique(fixture_list_3.as_matrix()[:, 0]),
                            ((pd.read_csv("../data/team_id_20192020.csv", header=0)).as_matrix())[:, 0]))

In [ ]:
teams = np.unique(team_list)

In [ ]:
fl1 = fixture_list_1.as_matrix()
fl2 = fixture_list_2.as_matrix()
fl3 = fixture_list_3.as_matrix()

In [ ]:
# test
a = np.ones(len(teams)) * 0.1
d = np.ones(len(teams)) * 0.1
mu = 0.1
l = likelihood_season(fl1, teams, mu, a, d)

In [ ]:
# test
predict_fixtures(fl1[1:3, :], teams, mu, a, d)

In [ ]:
priormeans = np.ones((2 * len(teams)) + 2) * 0.0
priorsds = np.ones((2 * len(teams)) + 2) * 0.1
beta = 0.001
niter = 50000

# prior-out promoted teams
inds = list(np.where(teams == 'Norwich City')[0])
inds.append(np.where(teams == 'Aston Villa')[0][0])
inds.append(np.where(teams == 'Sheffield United')[0][0])
inds = np.array(inds)

est = EstimateParameters(fl1, fl2, fl3, teams, beta, priormeans, priorsds, niter=niter, zerooutinds=inds)

In [ ]:
tm = 'Man City'
plot.plot(est[:, (1 + np.where(teams == tm)[0][0]).astype(int)], 'r')
plot.plot(est[:, (len(teams) + 1 + np.where(teams == tm)[0][0]).astype(int)], 'b')
plot.show()

plot.plot(est[:, 0], 'k')
plot.show()

#### Save parameters

In [ ]:
# save parameters
burn = int((2 * niter) / 5)
mu_mean = np.array([np.mean(est[burn:, 0])])
mu_sd = np.array([np.std(est[burn:, 0])])
a_mean = np.mean(est[burn:, 1:(len(teams) + 1)], axis=0)
a_sd = np.std(est[burn:, 1:(len(teams) + 1)], axis=0)
d_mean = np.mean(est[burn:, (len(teams) + 1):((2 * len(teams)) + 1)], axis=0)
d_sd = np.std(est[burn:, (len(teams) + 1):((2 * len(teams)) + 1)], axis=0)
means = np.concatenate((mu_mean, a_mean, d_mean))
sds = np.concatenate((mu_sd, a_sd, d_sd))

sds[1 + inds] = 0.1
sds[1 + len(teams) + inds] = 0.1

with open('../parameters/all_teams.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for tm in teams:
        csv_writer.writerow([tm])
csv_file.close()

with open('../parameters/all_teams_params.csv', mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for i in range(((2 * len(teams)) + 1)):
        csv_writer.writerow([means[i], sds[i]])
csv_file.close()

#### Simulate seasons performance of teams

In [ ]:
fixture_list_this_season = pd.read_csv("../data/prem_results_20192020.csv").values

wins = np.zeros(20)
topfour = np.zeros(20)
niter = 300

# predict lambdas - average form
muest = np.hstack((means[0], sds[0]))
aest = np.hstack((np.reshape(means[1:(1+len(teams))], ((len(teams), 1))),
                  np.reshape(sds[1:(1+len(teams))], ((len(teams), 1)))))
dest = np.hstack((np.reshape(means[(1+len(teams)):(1+(2*len(teams)))], ((len(teams), 1))),
                  np.reshape(sds[(1+len(teams)):(1+(2*len(teams)))], ((len(teams), 1)))))

for j in range(niter):
    # work out total goals predicted
    total_goals = np.zeros(20)
    #actual_goals = np.zeros(20)
    total_goals_conceded = np.zeros(20)
    #actual_goals_conceded = np.zeros(20)
    tms = np.unique(fixture_list_this_season[:, 0])
    goals_this_season = predict_fixtures(fixture_list_this_season, teams, muest, aest, dest, uncertainty=True)
    pnts = np.zeros(20)
    for i in range(np.shape(fixture_list_this_season)[0]):
        goals_ht = np.random.poisson(goals_this_season[0][i])
        goals_at = np.random.poisson(goals_this_season[1][i])
        total_goals[np.where(tms == fixture_list_this_season[i, 0])[0][0].astype(int)] += goals_ht
        total_goals[np.where(tms == fixture_list_this_season[i, 1])[0][0].astype(int)] += goals_at
        pnts[np.where(tms == fixture_list_this_season[i, 0])[0][0].astype(int)] += ((goals_at < goals_ht) * 3) + (goals_at == goals_ht)
        pnts[np.where(tms == fixture_list_this_season[i, 1])[0][0].astype(int)] += ((goals_at > goals_ht) * 3) + (goals_at == goals_ht)
        #actual_goals[np.where(tms == fl2[i, 0])[0][0].astype(int)] += fl2[i, 2]
        #actual_goals[np.where(tms == fl2[i, 1])[0][0].astype(int)] += fl2[i, 3]
        total_goals_conceded[np.where(tms == fixture_list_this_season[i, 0])[0][0].astype(int)] += goals_at
        total_goals_conceded[np.where(tms == fixture_list_this_season[i, 1])[0][0].astype(int)] += goals_ht
        #actual_goals_conceded[np.where(tms == fl2[i, 0])[0][0].astype(int)] += fl2[i, 3]
        #actual_goals_conceded[np.where(tms == fl2[i, 1])[0][0].astype(int)] += fl2[i, 2]
    # update title wins
    wins[np.argmax(pnts)] += 1
    # update top four finishes
    topfour[np.argsort(pnts)[16:]] += 1

In [ ]:
ind = np.arange(len(topfour))  # the x locations for the groups
width = 0.35  # the width of the bars

fig, ax = plot.subplots()
rects1 = ax.bar(ind - width/2, wins/niter, width,
                label='Wins')
rects2 = ax.bar(ind + width/2, topfour/niter, width,
                label='Top Four')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Probability')
ax.set_xticks(ind)
ax.set_xticklabels(tuple(tms))
ax.legend()

In [ ]:
print(tms)
print(wins/niter)

In [ ]:
print(tms)
print(total_goals)
print(total_goals_conceded)
print(total_goals - total_goals_conceded)
print(np.argsort(pnts))